In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/NLP Project/Dataset_Clean/Combined Data.csv')
data

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
53038,53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53043 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53043 non-null  int64 
 1   statement   52681 non-null  object
 2   status      53043 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [ ]:
data['status'].value_counts()

,count
status,
Normal,16351
Depression,15404
Suicidal,10653
Anxiety,3888
Bipolar,2877
Stress,2669
Personality disorder,1201


In [ ]:
ptsd = pd.read_csv('/content/drive/MyDrive/NLP Project/Dataset_Clean/ptsd_2019_features_tfidf_256.csv', encoding='ISO-8859-1')
ptsd.head(2)

,subreddit,author,date,post,automated_readability_index,coleman_liau_index,flesch_kincaid_grade_level,flesch_reading_ease,gulpease_index,gunning_fog_index,...,tfidf_wish,tfidf_without,tfidf_wonder,tfidf_work,tfidf_worri,tfidf_wors,tfidf_would,tfidf_wrong,tfidf_x200b,tfidf_year
0,ptsd,sjhachman,2019/01/01,Ashamed of my triggers So it's been 11 months ...,6.033023,6.348100,6.294585,79.470253,66.386935,8.643384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.107753,0.0,0.0,0.000000
1,ptsd,theprincessofpirates,2019/01/01,Help with letting go of trauma when it is actu...,5.302500,6.236102,6.190000,77.400000,68.166667,9.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.161208


In [ ]:
ptsd = ptsd[['subreddit', 'post']]

In [ ]:
ptsd.head(2)

,subreddit,post
0,ptsd,Ashamed of my triggers So it's been 11 months ...
1,ptsd,Help with letting go of trauma when it is actu...


In [ ]:
!pip install praw pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import praw

# Replace with your credentials
reddit = praw.Reddit(
    client_id="4fnldcuzfJ8gyenON_kAXA",
    client_secret="PwDuXUOZP9QdrBGLAcsM78on-cTLmw",
    user_agent="mental_health_dataset_collector"
)


In [ ]:
mental_health_subreddits = {
    "Depression": "depression",
    "Anxiety": "Anxiety",
    "Stress": "stress",
    "PTSD": "PTSD",
    "Suicidal": "SuicideWatch",
    "Loneliness": "loneliness",
    "Bipolar": "bipolar",
    "OCD": "OCD",
    "Schizophrenia": "schizophrenia",
    "ADHD": "ADHD",
    "EatingDisorder": "EatingDisorders",
    "Addiction": "addiction",
    "Normal": "CasualConversation"
}


In [ ]:
import re

def clean_text(text):
    text = re.sub(r'http\S+', '', text)               # Remove URLs
    text = re.sub(r'@\w+', '', text)                  # Remove mentions
    text = re.sub(r'#\w+', '', text)                  # Remove hashtags
    text = re.sub(r'\n+', ' ', text)                  # Remove newlines
    text = re.sub(r'\s+', ' ', text).strip()          # Remove extra spaces
    text = re.sub(r'[^\w\s.,!?]', '', text)           # Remove emojis/symbols
    return text


In [ ]:
import pandas as pd

all_data = []

for condition, subreddit_name in mental_health_subreddits.items():
    print(f"Scraping {condition} from r/{subreddit_name}")
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.top(limit=3000)  

    count = 0
    for post in posts:
        if count >= 2000:
            break
        text = post.title + " " + post.selftext
        text = clean_text(text)
        if len(text) > 50:  
            all_data.append({"condition": condition, "text": text})
            count += 1

    print(f"Collected {count} posts for {condition}")


In [ ]:
df = pd.DataFrame(all_data)
print(df.head())

# Save as CSV
df.to_csv("mental_health_dataset.csv", index=False)

# Download in Colab
from google.colab import files
files.download("mental_health_dataset.csv")


    condition                                               text
0  Depression  Shout out to the particular hell that is funct...
1  Depression  I hate that people dont understand that i dont...
2  Depression  If 10 years ago someone told me that in 10 yea...
3  Depression  Its like I died at 15, but my body just kept o...
4  Depression  Highfunctioning depression I feel like Im livi...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model Training Part


In [ ]:
!pip install transformers datasets scikit-learn torch

 Load and Inspect Dataset

In [ ]:
import pandas as pd

# Load the cleaned dataset you created earlier
df = pd.read_csv("/content/drive/MyDrive/NLP Project/Dataset_Clean/mental_health_dataset.csv")

# Inspect
print(df.head())
print(df['condition'].value_counts())


    condition                                               text
0  Depression  Shout out to the particular hell that is funct...
1  Depression  I hate that people dont understand that i dont...
2  Depression  If 10 years ago someone told me that in 10 yea...
3  Depression  Its like I died at 15, but my body just kept o...
4  Depression  Highfunctioning depression I feel like Im livi...
condition
EatingDisorder    996
ADHD              993
PTSD              990
Normal            984
Stress            976
Suicidal          967
Addiction         964
Depression        963
Anxiety           952
Loneliness        921
Schizophrenia     475
OCD               445
Bipolar           364
Name: count, dtype: int64


Over Sampling

In [ ]:
import pandas as pd
from sklearn.utils import resample

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP Project/Dataset_Clean/mental_health_dataset.csv")

In [ ]:
# Set target count per class
target_count = 1000

# Separate data by class
dfs = []
for label in df['condition'].unique():
    subset = df[df['condition'] == label]
    if len(subset) > target_count:
        # Downsample majority classes
        subset_down = resample(subset,
                               replace=False,
                               n_samples=target_count,
                               random_state=42)
        dfs.append(subset_down)
    elif len(subset) < target_count:
        # Upsample minority classes
        subset_up = resample(subset,
                             replace=True,
                             n_samples=target_count,
                             random_state=42)
        dfs.append(subset_up)
    else:
        dfs.append(subset)

In [ ]:
# Combine all into a balanced dataframe
df_balanced = pd.concat(dfs)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print(df_balanced['condition'].value_counts())

# Save balanced dataset
df_balanced.to_csv("mental_health_balanced.csv", index=False)

condition
PTSD              1000
Stress            1000
Loneliness        1000
ADHD              1000
Anxiety           1000
Schizophrenia     1000
OCD               1000
Suicidal          1000
Depression        1000
Normal            1000
EatingDisorder    1000
Addiction         1000
Bipolar           1000
Name: count, dtype: int64


In [ ]:
df = pd.read_csv("/content/mental_health_balanced.csv")
print(df.head())
print(df['condition'].value_counts())

    condition                                               text
0        PTSD  I am so fucking sad I dont know why Im writing...
1      Stress  When I get stressed at work I just want to hid...
2  Loneliness  why do i have to be always alone ? Im tired of...
3        ADHD  I just showered for the first time in A MONTH!...
4     Anxiety  I got a 4.0 this semester in college! I dont e...
condition
PTSD              1000
Stress            1000
Loneliness        1000
ADHD              1000
Anxiety           1000
Schizophrenia     1000
OCD               1000
Suicidal          1000
Depression        1000
Normal            1000
EatingDisorder    1000
Addiction         1000
Bipolar           1000
Name: count, dtype: int64


Encode condition Labels with LabelEncoder